In [1]:
import numpy as np
import tensorflow as tf
from utils import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from utils import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformation,padding
from keras.utils import Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform

Using TensorFlow backend.


In [2]:
epochs = 1000

In [3]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [4]:
trainSentences = readfile("../data/conll2003/en/raw/train.txt")
devSentences = readfile("../data/conll2003/en/raw//valid.txt")
testSentences = readfile("../data/conll2003/en/raw/test.txt")

FileNotFoundError: [Errno 2] No such file or directory: '../data/conll2003/en/raw/train.txt'

In [32]:
trainSentences = addCharInformation(trainSentences)
devSentences = addCharInformation(devSentences)
testSentences = addCharInformation(testSentences)

In [33]:
labelSet = set()
words = {}

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token,char,label in sentence:
            labelSet.add(label)
            words[token.lower()] = True

# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

# :: Read in word embeddings ::
word2Idx = {}
wordEmbeddings = []

fEmbeddings = open("glove.6B.100d.txt")#, encoding="utf-8")  

In [34]:
for line in fEmbeddings:
    split = line.strip().split(" ")
    word = split[0]
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(len(split)-1) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)
        
        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, len(split)-1)
        wordEmbeddings.append(vector)

    if split[0].lower() in words:
        vector = np.array([float(num) for num in split[1:]])
        wordEmbeddings.append(vector)
        word2Idx[split[0]] = len(word2Idx)
        
wordEmbeddings = np.array(wordEmbeddings) #GLOVE EMBEDDINGS

In [35]:
char2Idx = {"PADDING":0, "UNKNOWN":1}
characterVariety = " 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ.,-_()[]{}!?:;#'\"/\\%$`&=*+@^~|"
for c in characterVariety:
    char2Idx[c] = len(char2Idx)

In [36]:
train_set = padding(createMatrices(trainSentences,word2Idx,  label2Idx, case2Idx,char2Idx))
dev_set = padding(createMatrices(devSentences,word2Idx, label2Idx, case2Idx,char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx,char2Idx))

idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

# Neural Network

In [37]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')

"""
#TODO: change to Contextual String Embeddings here from Flair Paper
#TODO: Instead of Concatenating character-based word representations perform Attending to character-based representations
#for rare words http://www.marekrei.com/blog/attending-to-characters-in-neural-sequence-labeling-models/
"""

embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5)(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52))(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
output = concatenate([words, casing,char]) #concatenations of words and char
output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.25))(output)
output = TimeDistributed(Dense(len(label2Idx), activation='softmax'))(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam')
model.summary()

#TODO add CRF here
#plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 30) 2850        char_input[0][0]                 
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, None, 52, 30) 0           char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_9 (TimeDistrib (None, None, 52, 30) 2730        dropout_5[0][0]                  
__________________________________________________________________________________________________
time_distr

In [38]:
for epoch in range(epochs):    
    print("Epoch %d/%d"%(epoch,epochs))
    a = Progbar(len(train_batch_len))
    for i,batch in enumerate(iterate_minibatches(train_batch,train_batch_len)):
        labels, tokens, casing,char = batch       
        model.train_on_batch([tokens, casing,char], labels)
        a.update(i)
    print(' ')

Epoch 0/1000
63/64 [============================>.] - ETA: 0s 
Epoch 1/1000
63/64 [============================>.] - ETA: 0s 
Epoch 2/1000
63/64 [============================>.] - ETA: 0s 
Epoch 3/1000
63/64 [============================>.] - ETA: 0s 
Epoch 4/1000
63/64 [============================>.] - ETA: 0s 
Epoch 5/1000
63/64 [============================>.] - ETA: 0s 
Epoch 6/1000
63/64 [============================>.] - ETA: 0s 
Epoch 7/1000
63/64 [============================>.] - ETA: 0s 
Epoch 8/1000
63/64 [============================>.] - ETA: 0s 
Epoch 9/1000
63/64 [============================>.] - ETA: 0s 
Epoch 10/1000
63/64 [============================>.] - ETA: 0s 
Epoch 11/1000
63/64 [============================>.] - ETA: 0s 
Epoch 12/1000
63/64 [============================>.] - ETA: 0s 
Epoch 13/1000
63/64 [============================>.] - ETA: 0s 
Epoch 14/1000
63/64 [============================>.] - ETA: 0s 
Epoch 15/1000
63/64 [============================>

63/64 [============================>.] - ETA: 0s 
Epoch 254/1000
63/64 [============================>.] - ETA: 0s 
Epoch 255/1000
63/64 [============================>.] - ETA: 0s 
Epoch 256/1000
63/64 [============================>.] - ETA: 0s 
Epoch 257/1000
63/64 [============================>.] - ETA: 0s 
Epoch 258/1000
63/64 [============================>.] - ETA: 0s 
Epoch 259/1000
63/64 [============================>.] - ETA: 0s 
Epoch 260/1000
63/64 [============================>.] - ETA: 0s 
Epoch 261/1000
63/64 [============================>.] - ETA: 0s 
Epoch 262/1000
63/64 [============================>.] - ETA: 0s 
Epoch 263/1000
63/64 [============================>.] - ETA: 0s 
Epoch 264/1000
63/64 [============================>.] - ETA: 0s 
Epoch 265/1000
63/64 [============================>.] - ETA: 0s 
Epoch 266/1000
63/64 [============================>.] - ETA: 0s 
Epoch 267/1000
63/64 [============================>.] - ETA: 0s 
Epoch 268/1000
63/64 [==================

63/64 [============================>.] - ETA: 0s 
Epoch 506/1000
63/64 [============================>.] - ETA: 0s 
Epoch 507/1000
63/64 [============================>.] - ETA: 0s 
Epoch 508/1000
63/64 [============================>.] - ETA: 0s 
Epoch 509/1000
63/64 [============================>.] - ETA: 0s 
Epoch 510/1000
63/64 [============================>.] - ETA: 0s 
Epoch 511/1000
63/64 [============================>.] - ETA: 0s 
Epoch 512/1000
63/64 [============================>.] - ETA: 0s 
Epoch 513/1000
63/64 [============================>.] - ETA: 0s 
Epoch 514/1000
63/64 [============================>.] - ETA: 0s 
Epoch 515/1000
63/64 [============================>.] - ETA: 0s 
Epoch 516/1000
63/64 [============================>.] - ETA: 0s 
Epoch 517/1000
63/64 [============================>.] - ETA: 0s 
Epoch 518/1000
63/64 [============================>.] - ETA: 0s 
Epoch 519/1000
63/64 [============================>.] - ETA: 0s 
Epoch 520/1000
63/64 [==================

63/64 [============================>.] - ETA: 0s 
Epoch 758/1000
63/64 [============================>.] - ETA: 0s 
Epoch 759/1000
63/64 [============================>.] - ETA: 0s 
Epoch 760/1000
63/64 [============================>.] - ETA: 0s 
Epoch 761/1000
63/64 [============================>.] - ETA: 0s 
Epoch 762/1000
63/64 [============================>.] - ETA: 0s 
Epoch 763/1000
63/64 [============================>.] - ETA: 0s 
Epoch 764/1000
63/64 [============================>.] - ETA: 0s 
Epoch 765/1000
63/64 [============================>.] - ETA: 0s 
Epoch 766/1000
63/64 [============================>.] - ETA: 0s 
Epoch 767/1000
63/64 [============================>.] - ETA: 0s 
Epoch 768/1000
63/64 [============================>.] - ETA: 0s 
Epoch 769/1000
63/64 [============================>.] - ETA: 0s 
Epoch 770/1000
63/64 [============================>.] - ETA: 0s 
Epoch 771/1000
63/64 [============================>.] - ETA: 0s 
Epoch 772/1000
63/64 [==================

In [39]:
# For 1000 epochs
#   Performance on dev dataset        
predLabels, correctLabels = tag_dataset(dev_batch)        
pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))
    
#   Performance on test dataset       
predLabels, correctLabels = tag_dataset(test_batch)        
pre_test, rec_test, f1_test= compute_f1(predLabels, correctLabels, idx2Label)
print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_test, rec_test, f1_test))

3451/3453 [============================>.] - ETA: 0sTest-Data: Prec: 0.894, Rec: 0.905, F1: 0.900


In [ ]:
#Try Pydense CRF here

In [23]:
#KOMINOS EMBEDDINGS
import tarfile, gzip
import wget
"""Extended Dependency Based Skip-gram"""

#https://github.com/UKPLab/emnlp2017-bilstm-cnn-crf/blob/master/Train_Chunking.py
#From TU Darmstadt
def KOMNINOSEmbedding():
    base_path ='https://public.ukp.informatik.tu-darmstadt.de/reimers/embeddings/komninos_english_embeddings.gz'
    wget.download(base_path)
    wget = 
    unziped = gzip.open('komninos_english_embeddings.gz', 'rb')
    file_content = unziped.read()
    print (file_content.shape)
    unziped.close()
    return file_content

In [24]:
KOMNINOSEmbedding()

AttributeError: 'str' object has no attribute 'shape'

In [38]:
#Contextual String Embedding
#Load the Forward and Backward Model
"""Contextual string embeddings of words, as proposed in Akbik et al., 2018."""
def ContextualStringEmbedding(type):
    if type == 'news-forward':
        base_path = 'https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-forward.pt'
        wget.download(base_path)
    if type == 'news-backward':
        base_path = 'https://s3.eu-central-1.amazonaws.com/alan-nlp/resources/embeddings/lm-news-english-backward.pt'
        wget.download(base_path)
    return None

In [39]:
ContextualStringEmbedding('news-forward')

In [ ]:
#TODO Train 1 Billion word model for the language model

